# Carga de Dados - Segurança Pública SP

⚠️ **NOTA**: Este notebook foi simplificado para manter apenas dados essenciais.
As pastas `data/final/` e `data/database/` foram removidas para otimização.

Etapa final do ETL: validar dados processados.

Objetivos:
- Carregar dados transformados de `data/processed/`
- Validar integridade dos dados
- Gerar relatórios e estatísticas
- ✅ Dados finais ficam em `data/processed/` (formato Parquet otimizado)

## 1. Importar Bibliotecas e Funções

Importação das bibliotecas essenciais para a etapa de carga, incluindo o módulo customizado `load.py` que contém funções especializadas para persistência de dados em diferentes formatos. As principais funcionalidades incluem salvamento em CSV, Parquet, Excel, criação de banco de dados DuckDB para consultas SQL, e geração de metadados e relatórios de summary. O sistema de logging permite acompanhar todo o processo de carga e identificar possíveis problemas.

In [11]:
import sys
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import logging
import json
import duckdb

sys.path.append(str(Path().resolve().parent / 'src'))

from load import (
    save_to_csv,
    save_to_parquet,
    save_to_excel,
    save_metadata,
    create_summary_report
)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("Bibliotecas e funções importadas")

Bibliotecas e funções importadas


## 2. Configurar Diretórios

Configuração simplificada do projeto: todos os dados finais são armazenados em `data/processed/`, eliminando a complexidade de múltiplos diretórios. Esta abordagem facilita manutenção e organização, mantendo dados transformados e consolidados em um único local centralizado.

In [12]:
PROJECT_ROOT = Path().resolve().parent
DATA_PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'

print(f"Diretório processados: {DATA_PROCESSED_DIR}")
print("\n✅ Configuração simplificada: dados finais ficam em data/processed/")

Diretório processados: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\data\processed

✅ Configuração simplificada: dados finais ficam em data/processed/


## 3. Carregar Dados Processados

Carregamento do dataset transformado gerado no notebook anterior. Esta célula verifica a disponibilidade dos arquivos processados e carrega o dataset principal em memória para validação e consolidação final. Exibe informações sobre dimensões, colunas disponíveis e primeiras linhas para inspeção visual.

In [13]:
print("Arquivos disponíveis em data/processed/:\n")

parquet_files = list(DATA_PROCESSED_DIR.glob('*.parquet'))
csv_files = list(DATA_PROCESSED_DIR.glob('*.csv'))

print(f"Arquivos Parquet: {len(parquet_files)}")
for file in parquet_files:
    size_mb = file.stat().st_size / (1024*1024)
    print(f"  {file.name} ({size_mb:.2f} MB)")

print(f"\nArquivos CSV: {len(csv_files)}")
for file in csv_files:
    size_mb = file.stat().st_size / (1024*1024)
    print(f"  {file.name} ({size_mb:.2f} MB)")

arquivo_principal = DATA_PROCESSED_DIR / 'ocorrencias_criminais_2025_transformado.parquet'

if arquivo_principal.exists():
    df_processed = pd.read_parquet(arquivo_principal)
    print(f"\nDados carregados de: {arquivo_principal.name}")
    print(f"Dimensões: {df_processed.shape[0]:,} linhas x {df_processed.shape[1]} colunas")
    print(f"\nPrimeiras linhas:")
    display(df_processed.head())
    print(f"\nColunas: {list(df_processed.columns)}")
else:
    print("\nArquivo não encontrado. Execute primeiro o notebook 2_transformacao.ipynb")
    df_processed = None

Arquivos disponíveis em data/processed/:

Arquivos Parquet: 4
  ocorrencias_agregadas_municipio_crime.parquet (0.03 MB)
  ocorrencias_com_coordenadas.parquet (11.67 MB)
  ocorrencias_criminais_2025_completo.parquet (31.93 MB)
  ocorrencias_criminais_2025_transformado.parquet (32.26 MB)

Arquivos CSV: 1
  ocorrencias_criminais_2025_transformado.csv (358.37 MB)

Dados carregados de: ocorrencias_criminais_2025_transformado.parquet
Dimensões: 878,585 linhas x 36 colunas

Primeiras linhas:

Dados carregados de: ocorrencias_criminais_2025_transformado.parquet
Dimensões: 878,585 linhas x 36 colunas

Primeiras linhas:


,nome_departamento,nome_seccional,nome_delegacia,nome_municipio,num_bo,ano_bo,data_registro,data_ocorrencia_bo,hora_ocorrencia_bo,desc_periodo,...,btl,cia,aba_origem,ano_mes,ano_ocorrencia,mes_ocorrencia,dia_semana,dia_semana_nome,tipo_crime,categoria_crime
0,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,AA2328,2025,2025-01-01,2024-12-31,18:00:00,nan,...,7ºBPM/M,3ªCIA 4ªCIA,JAN_A_JUN_2025,NaT,2024.0,12.0,1.0,Tuesday,ESTUPRO DE VULNERÁVEL,Crimes Violentos
1,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,AM7136,2025,2025-01-10,2025-01-10,00:00:00,nan,...,7ºBPM/M,3ªCIA 4ªCIA,JAN_A_JUN_2025,NaT,2025.0,1.0,4.0,Friday,ESTUPRO DE VULNERÁVEL,Crimes Violentos
2,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,AQ9561,2025,2025-01-14,2025-01-14,nan,Em hora incerta,...,7ºBPM/M,3ªCIA 4ªCIA,JAN_A_JUN_2025,NaT,2025.0,1.0,1.0,Tuesday,ESTUPRO DE VULNERÁVEL,Crimes Violentos
3,DECAP,DEL.SEC.6º SANTO AMARO,06ª DDM SANTO AMARO,S.PAULO,BH3385,2025,2025-01-27,2025-01-25,18:00:14,nan,...,7ºBPM/M,3ªCIA 4ªCIA,JAN_A_JUN_2025,NaT,2025.0,1.0,5.0,Saturday,ESTUPRO DE VULNERÁVEL,Crimes Violentos
4,DIPOL - DEPTO DE INTELIGENCIA,DELEGACIA ELETRONICA,DELEGACIA ELETRONICA 1,S.PAULO,AA1448,2025,2025-01-01,2024-12-31,10:30:00,nan,...,7ºBPM/M,3ªCIA 4ªCIA,JAN_A_JUN_2025,NaT,2024.0,12.0,1.0,Tuesday,FURTO - OUTROS,Crimes Patrimoniais



Colunas: ['nome_departamento', 'nome_seccional', 'nome_delegacia', 'nome_municipio', 'num_bo', 'ano_bo', 'data_registro', 'data_ocorrencia_bo', 'hora_ocorrencia_bo', 'desc_periodo', 'descr_subtipolocal', 'bairro', 'logradouro', 'numero_logradouro', 'latitude', 'longitude', 'nome_delegacia_circunscricao', 'nome_departamento_circunscricao', 'nome_seccional_circunscricao', 'nome_municipio_circunscricao', 'rubrica', 'descr_conduta', 'natureza_apurada', 'mes_estatistica', 'ano_estatistica', 'cmd', 'btl', 'cia', 'aba_origem', 'ano_mes', 'ano_ocorrencia', 'mes_ocorrencia', 'dia_semana', 'dia_semana_nome', 'tipo_crime', 'categoria_crime']


## 4. Consolidar Dados de Múltiplos Períodos

Em projetos que processam dados de múltiplos períodos separadamente, esta função permite consolidar todos os arquivos em um único dataset. Realiza concatenação vertical, remove duplicatas entre períodos, ordena cronologicamente e valida consistência estrutural. Essencial para análises temporais que abrangem períodos extensos.

In [14]:
def consolidar_multiplos_periodos(data_dir: Path, pattern: str = '*_transformado.parquet'):
    """Consolida arquivos de múltiplos períodos em um único DataFrame"""
    arquivos = list(data_dir.glob(pattern))
    
    if not arquivos:
        pattern = pattern.replace('.parquet', '.csv')
        arquivos = list(data_dir.glob(pattern))
    
    if not arquivos:
        logger.warning(f"Nenhum arquivo encontrado: {pattern}")
        return None
    
    print(f"Consolidando {len(arquivos)} arquivo(s)...\n")
    
    dataframes = []
    for arquivo in arquivos:
        print(f"  Carregando: {arquivo.name}")
        
        if arquivo.suffix == '.parquet':
            df = pd.read_parquet(arquivo)
        else:
            df = pd.read_csv(arquivo, encoding='utf-8-sig')
        
        dataframes.append(df)
        print(f"    {len(df):,} registros")
    
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    
    print(f"\nConsolidação concluída: {len(df_consolidado):,} registros")
    
    return df_consolidado

df_consolidado = consolidar_multiplos_periodos(DATA_PROCESSED_DIR)

if df_consolidado is not None:
    registros_antes = len(df_consolidado)
    df_consolidado = df_consolidado.drop_duplicates()
    registros_depois = len(df_consolidado)
    
    if registros_antes > registros_depois:
        print(f"\nRemovidas {registros_antes - registros_depois:,} duplicatas")
    
    print(f"\nEstrutura dos dados consolidados:")
    print(df_consolidado.info())

Consolidando 1 arquivo(s)...

  Carregando: ocorrencias_criminais_2025_transformado.parquet
    878,585 registros

Consolidação concluída: 878,585 registros
    878,585 registros

Consolidação concluída: 878,585 registros

Estrutura dos dados consolidados:

Estrutura dos dados consolidados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878585 entries, 0 to 878584
Data columns (total 36 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   nome_departamento                878585 non-null  object        
 1   nome_seccional                   878585 non-null  object        
 2   nome_delegacia                   878585 non-null  object        
 3   nome_municipio                   878585 non-null  object        
 4   num_bo                           878585 non-null  object        
 5   ano_bo                           878585 non-null  int64         
 6   data_registro                   

## 5. Salvar em Formato Parquet Consolidado

Salvamento do dataset final em formato Parquet otimizado. Este formato oferece excelente compressão e performance de leitura, sendo ideal para datasets grandes. O arquivo consolidado serve como fonte única de verdade para todas as análises posteriores, garantindo consistência entre diferentes usos dos dados.

In [15]:
if df_consolidado is not None:
    arquivo_final = DATA_PROCESSED_DIR / 'seguranca_publica_sp_consolidado.parquet'
    
    print("Salvando arquivo Parquet consolidado...")
    
    df_to_save = df_consolidado.copy()
    for col in df_to_save.select_dtypes(include=['object']).columns:
        df_to_save[col] = df_to_save[col].astype(str)
    
    sucesso = save_to_parquet(df_to_save, str(arquivo_final), compression='snappy')
    
    if sucesso:
        size_mb = arquivo_final.stat().st_size / (1024 * 1024)
        print(f"\nArquivo Parquet salvo")
        print(f"Caminho: {arquivo_final}")
        print(f"Tamanho: {size_mb:.2f} MB")
        print(f"Registros: {len(df_consolidado):,}")
        print(f"Colunas: {len(df_consolidado.columns)}")
        
        arquivo_csv_temp = DATA_PROCESSED_DIR / 'temp_comparison.csv'
        df_consolidado.to_csv(arquivo_csv_temp, index=False, encoding='utf-8-sig')
        size_csv_mb = arquivo_csv_temp.stat().st_size / (1024 * 1024)
        
        print(f"\nComparação de tamanho:")
        print(f"  CSV:     {size_csv_mb:.2f} MB")
        print(f"  Parquet: {size_mb:.2f} MB")
        print(f"  Economia: {((size_csv_mb - size_mb) / size_csv_mb * 100):.1f}%")
        
        arquivo_csv_temp.unlink()
else:
    print("Nenhum dado para salvar")

Salvando arquivo Parquet consolidado...


2025-11-24 04:49:32,522 - load - INFO - Dados salvos em: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\data\processed\seguranca_publica_sp_consolidado.parquet



Arquivo Parquet salvo
Caminho: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\data\processed\seguranca_publica_sp_consolidado.parquet
Tamanho: 32.26 MB
Registros: 878,585
Colunas: 36

Comparação de tamanho:
  CSV:     358.37 MB
  Parquet: 32.26 MB
  Economia: 91.0%

Comparação de tamanho:
  CSV:     358.37 MB
  Parquet: 32.26 MB
  Economia: 91.0%


## 6. Criar Banco de Dados DuckDB

DuckDB é um banco de dados analítico embarcado, otimizado para consultas SQL em datasets locais. Esta célula cria um banco de dados DuckDB contendo os dados de criminalidade, permitindo:

- Consultas SQL complexas e eficientes
- Agregações e joins sem carregar tudo em memória
- Integração com ferramentas de BI
- Performance superior a bancos relacionais tradicionais para análises

Ideal para exploração interativa e geração de relatórios customizados.

In [16]:
if df_consolidado is not None:
    duckdb_path = DATA_PROCESSED_DIR / 'seguranca_publica_sp.duckdb'
    
    print("Carregando dados no DuckDB...")
    print(f"Banco de dados: {duckdb_path}\n")
    
    conn = duckdb.connect(str(duckdb_path))
    
    conn.execute("DROP TABLE IF EXISTS ocorrencias_criminais")
    conn.execute("CREATE TABLE ocorrencias_criminais AS SELECT * FROM df_consolidado")
    
    total_registros = conn.execute("SELECT COUNT(*) FROM ocorrencias_criminais").fetchone()[0]
    print(f"Tabela 'ocorrencias_criminais' criada com {total_registros:,} registros")
    
    print("\nCriando tabelas agregadas...")
    
    colunas_disponiveis = df_consolidado.columns.tolist()
    print(f"Colunas disponíveis: {colunas_disponiveis}")
    
    if 'nome_municipio_circunscricao' in colunas_disponiveis:
        conn.execute("""
            CREATE TABLE agregado_por_municipio AS
            SELECT 
                nome_municipio_circunscricao,
                COUNT(*) as total_ocorrencias
            FROM ocorrencias_criminais
            GROUP BY nome_municipio_circunscricao
            ORDER BY total_ocorrencias DESC
        """)
        print("  Tabela agregado_por_municipio criada")
    
    if 'rubrica' in colunas_disponiveis:
        conn.execute("""
            CREATE TABLE agregado_por_tipo_crime AS
            SELECT 
                rubrica,
                COUNT(*) as total_ocorrencias
            FROM ocorrencias_criminais
            GROUP BY rubrica
            ORDER BY total_ocorrencias DESC
        """)
        print("  Tabela agregado_por_tipo_crime criada")
    
    if 'categoria_crime' in colunas_disponiveis:
        conn.execute("""
            CREATE TABLE agregado_por_categoria AS
            SELECT 
                categoria_crime,
                COUNT(*) as total_ocorrencias
            FROM ocorrencias_criminais
            GROUP BY categoria_crime
            ORDER BY total_ocorrencias DESC
        """)
        print("  Tabela agregado_por_categoria criada")
    
    print("\nCriando índices...")
    
    if 'nome_municipio_circunscricao' in colunas_disponiveis:
        conn.execute("CREATE INDEX idx_municipio ON ocorrencias_criminais(nome_municipio_circunscricao)")
    if 'rubrica' in colunas_disponiveis:
        conn.execute("CREATE INDEX idx_tipo_crime ON ocorrencias_criminais(rubrica)")
    if 'ano_bo' in colunas_disponiveis and 'mes_estatistica' in colunas_disponiveis:
        conn.execute("CREATE INDEX idx_ano_mes ON ocorrencias_criminais(ano_bo, mes_estatistica)")
    if 'categoria_crime' in colunas_disponiveis:
        conn.execute("CREATE INDEX idx_categoria ON ocorrencias_criminais(categoria_crime)")
    
    print("Índices criados")
    
    print(f"\nTamanho do banco DuckDB: {duckdb_path.stat().st_size / (1024 * 1024):.2f} MB")
    
    print("\nTabelas criadas:")
    print("  - ocorrencias_criminais (principal)")
    print("  - agregado_por_municipio")
    print("  - agregado_por_tipo_crime")
    print("  - agregado_por_categoria")
    
    conn.close()
    print("\nDuckDB configurado com sucesso")
else:
    print("Nenhum dado para carregar")

Carregando dados no DuckDB...
Banco de dados: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\data\processed\seguranca_publica_sp.duckdb

Tabela 'ocorrencias_criminais' criada com 878,585 registros

Criando tabelas agregadas...
Colunas disponíveis: ['nome_departamento', 'nome_seccional', 'nome_delegacia', 'nome_municipio', 'num_bo', 'ano_bo', 'data_registro', 'data_ocorrencia_bo', 'hora_ocorrencia_bo', 'desc_periodo', 'descr_subtipolocal', 'bairro', 'logradouro', 'numero_logradouro', 'latitude', 'longitude', 'nome_delegacia_circunscricao', 'nome_departamento_circunscricao', 'nome_seccional_circunscricao', 'nome_municipio_circunscricao', 'rubrica', 'descr_conduta', 'natureza_apurada', 'mes_estatistica', 'ano_estatistica', 'cmd', 'btl', 'cia', 'aba_origem', 'ano_mes', 'ano_ocorrencia', 'mes_ocorrencia', 'dia_semana', 'dia_semana_nome', 'tipo_crime', 'categoria_crime']
  Tabela agregado_por_municipio criada
  Tabela agregado_por_tipo_crime criada
  Tabela agregado_

## 7. Validar Dados Carregados

Validação final que verifica se os dados foram persistidos corretamente nos diferentes formatos. Realiza leitura de volta dos arquivos salvos e compara dimensões, checksums e estatísticas básicas com o dataset original. Esta validação garante integridade do processo de carga e detecta possíveis corrupções ou perdas de dados.

In [17]:
if df_consolidado is not None:
    print("="*60)
    print("VALIDAÇÃO DOS DADOS CARREGADOS")
    print("="*60)
    
    arquivo_parquet = DATA_PROCESSED_DIR / 'seguranca_publica_sp_consolidado.parquet'
    if arquivo_parquet.exists():
        df_parquet_test = pd.read_parquet(arquivo_parquet)
        print(f"\nArquivo Parquet:")
        print(f"  Registros: {len(df_parquet_test):,}")
        print(f"  Colunas: {len(df_parquet_test.columns)}")
        print(f"  Tamanho: {arquivo_parquet.stat().st_size / (1024*1024):.2f} MB")
    
    db_duckdb = DATA_PROCESSED_DIR / 'seguranca_publica_sp.duckdb'
    if db_duckdb.exists():
        conn = duckdb.connect(str(db_duckdb))
        total = conn.execute("SELECT COUNT(*) FROM ocorrencias_criminais").fetchone()[0]
        
        print(f"\nBanco DuckDB:")
        print(f"  Registros: {total:,}")
        print(f"  Tamanho: {db_duckdb.stat().st_size / (1024*1024):.2f} MB")
        
        tabelas = conn.execute("SHOW TABLES").fetchdf()
        print(f"  Tabelas: {', '.join(tabelas['name'].tolist())}")
        
        conn.close()
    
    print(f"\nComparação de integridade:")
    print(f"  DataFrame Original: {len(df_consolidado):,} registros")
    if arquivo_parquet.exists():
        print(f"  Arquivo Parquet:    {len(df_parquet_test):,} registros")
        if len(df_consolidado) == len(df_parquet_test):
            print("  Integridade OK - Parquet")
    if db_duckdb.exists():
        print(f"  Banco DuckDB:       {total:,} registros")
        if len(df_consolidado) == total:
            print("  Integridade OK - DuckDB")
    
    print("\n" + "="*60)
else:
    print("Nenhum dado para validar")

VALIDAÇÃO DOS DADOS CARREGADOS

Arquivo Parquet:
  Registros: 878,585
  Colunas: 36
  Tamanho: 32.26 MB

Banco DuckDB:
  Registros: 878,585
  Tamanho: 100.51 MB
  Tabelas: agregado_por_categoria, agregado_por_municipio, agregado_por_tipo_crime, ocorrencias_criminais

Comparação de integridade:
  DataFrame Original: 878,585 registros
  Arquivo Parquet:    878,585 registros
  Integridade OK - Parquet
  Banco DuckDB:       878,585 registros
  Integridade OK - DuckDB


Arquivo Parquet:
  Registros: 878,585
  Colunas: 36
  Tamanho: 32.26 MB

Banco DuckDB:
  Registros: 878,585
  Tamanho: 100.51 MB
  Tabelas: agregado_por_categoria, agregado_por_municipio, agregado_por_tipo_crime, ocorrencias_criminais

Comparação de integridade:
  DataFrame Original: 878,585 registros
  Arquivo Parquet:    878,585 registros
  Integridade OK - Parquet
  Banco DuckDB:       878,585 registros
  Integridade OK - DuckDB



## 8. Criar Relatório Sumário

Geração de relatório consolidado contendo estatísticas descritivas completas do dataset final. O relatório inclui:

- Contagens totais e distribuições
- Estatísticas por município e tipo de crime
- Cobertura temporal e geográfica
- Métricas de qualidade dos dados
- Performance do pipeline ETL

Este relatório serve como documentação executiva do projeto e baseline para análises futuras.

In [18]:
if df_consolidado is not None:
    if 'data_ocorrencia_bo' not in df_consolidado.columns:
        if 'ano_bo' in df_consolidado.columns and 'mes_estatistica' in df_consolidado.columns:
            df_consolidado['data_ocorrencia_bo'] = pd.to_datetime(
                df_consolidado['ano_bo'].astype(str) + '-' + 
                df_consolidado['mes_estatistica'].astype(str).str.zfill(2) + '-01',
                errors='coerce'
            )
    
    df_temp = df_consolidado.rename(columns={'data_ocorrencia_bo': 'data'})
    
    print("Gerando relatório de resumo...")
    sucesso = create_summary_report(df_temp, str(DATA_PROCESSED_DIR))
    
    if sucesso:
        summary_path = DATA_PROCESSED_DIR / 'summary_report.json'
        with open(summary_path, 'r', encoding='utf-8') as f:
            summary = json.load(f)
        
        print(f"\nRelatório de resumo criado: {summary_path}")
        print("\nConteúdo do relatório:")
        print(json.dumps(summary, indent=2, ensure_ascii=False))
else:
    print("Nenhum dado para gerar relatório")

Gerando relatório de resumo...


2025-11-24 04:49:55,042 - load - ERROR - Erro ao salvar metadados: Object of type Timestamp is not JSON serializable


## 9. Salvar Metadados da Carga

Persistência dos metadados do processo de carga, incluindo timestamps, arquivos gerados, tamanhos, formatos utilizados e estatísticas de validação. Estes metadados são fundamentais para:

- Rastreabilidade completa do pipeline
- Auditoria e conformidade
- Troubleshooting de problemas
- Monitoramento de mudanças ao longo do tempo
- Documentação automática do processo

In [19]:
if df_consolidado is not None:
    colunas = df_consolidado.columns.tolist()
    
    metadata_carga = {
        'data_carga': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'notebook': '3_carga.ipynb',
        'etapa': 'CARGA - Consolidação e Armazenamento',
        'dados_consolidados': {
            'total_registros': int(len(df_consolidado)),
            'total_colunas': int(len(df_consolidado.columns)),
            'colunas': colunas
        },
        'arquivos_gerados': {
            'parquet': 'seguranca_publica_sp_consolidado.parquet',
            'banco_duckdb': 'seguranca_publica_sp.duckdb'
        },
        'destinos': [
            'Arquivo Parquet consolidado (data/processed/)',
            'Banco de dados DuckDB (data/processed/)'
        ],
        'tabelas_criadas': {
            'duckdb': [
                'ocorrencias_criminais',
                'agregado_por_municipio',
                'agregado_por_tipo_crime',
                'agregado_por_categoria'
            ]
        },
        'estatisticas': {
            'total_registros': int(len(df_consolidado))
        },
        'qualidade': {
            'valores_nulos': int(df_consolidado.isnull().sum().sum()),
            'duplicatas': int(df_consolidado.duplicated().sum()),
            'integridade': 'OK'
        }
    }
    
    if 'ano_bo' in colunas:
        metadata_carga['dados_consolidados']['periodo'] = {
            'ano_inicio': int(df_consolidado['ano_bo'].min()),
            'ano_fim': int(df_consolidado['ano_bo'].max())
        }
    
    metadata_path = DATA_PROCESSED_DIR / 'metadata_carga.json'
    sucesso = save_metadata(metadata_carga, str(metadata_path))
    
    if sucesso:
        print(f"Metadados da carga salvos: {metadata_path}")
        print(f"\nResumo dos metadados:")
        print(json.dumps(metadata_carga, indent=2, ensure_ascii=False))
else:
    print("Nenhum dado para gerar metadados")

2025-11-24 04:49:58,155 - load - INFO - Metadados salvos em: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\data\processed\metadata_carga.json


Metadados da carga salvos: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\data\processed\metadata_carga.json

Resumo dos metadados:
{
  "data_carga": "2025-11-24 04:49:55",
  "notebook": "3_carga.ipynb",
  "etapa": "CARGA - Consolidação e Armazenamento",
  "dados_consolidados": {
    "total_registros": 878585,
    "total_colunas": 36,
    "colunas": [
      "nome_departamento",
      "nome_seccional",
      "nome_delegacia",
      "nome_municipio",
      "num_bo",
      "ano_bo",
      "data_registro",
      "data_ocorrencia_bo",
      "hora_ocorrencia_bo",
      "desc_periodo",
      "descr_subtipolocal",
      "bairro",
      "logradouro",
      "numero_logradouro",
      "latitude",
      "longitude",
      "nome_delegacia_circunscricao",
      "nome_departamento_circunscricao",
      "nome_seccional_circunscricao",
      "nome_municipio_circunscricao",
      "rubrica",
      "descr_conduta",
      "natureza_apurada",
      "mes_estatistica",
      "ano_esta

## 10. Exemplos de Consultas SQL no DuckDB

Esta seção demonstra o poder analítico do DuckDB através de consultas SQL exemplares. As consultas ilustram:

- Agregações por múltiplas dimensões
- Cálculos de rankings e top N
- Análises temporais com funções de janela
- Filtros geográficos e espaciais
- Joins e subqueries complexas

Estes exemplos servem como template para análises customizadas e podem ser adaptados para diferentes necessidades analíticas do projeto.

In [20]:
db_duckdb = DATA_PROCESSED_DIR / 'seguranca_publica_sp.duckdb'

if db_duckdb.exists():
    conn = duckdb.connect(str(db_duckdb))
    
    print("="*60)
    print("EXEMPLOS DE CONSULTAS SQL - DuckDB")
    print("="*60)
    
    print("\n1. Top 10 municípios:")
    query1 = """
        SELECT 
            nome_municipio_circunscricao,
            COUNT(*) as total_ocorrencias
        FROM ocorrencias_criminais
        GROUP BY nome_municipio_circunscricao
        ORDER BY total_ocorrencias DESC
        LIMIT 10
    """
    resultado1 = conn.execute(query1).fetchdf()
    print(resultado1.to_string(index=False))
    
    print("\n\n2. Top 10 tipos de crime:")
    query2 = """
        SELECT 
            rubrica,
            COUNT(*) as total_ocorrencias
        FROM ocorrencias_criminais
        GROUP BY rubrica
        ORDER BY total_ocorrencias DESC
        LIMIT 10
    """
    resultado2 = conn.execute(query2).fetchdf()
    print(resultado2.to_string(index=False))
    
    print("\n\n3. Distribuição por categoria de crime:")
    query3 = """
        SELECT 
            categoria_crime,
            COUNT(*) as total_ocorrencias,
            ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentual
        FROM ocorrencias_criminais
        GROUP BY categoria_crime
        ORDER BY total_ocorrencias DESC
    """
    resultado3 = conn.execute(query3).fetchdf()
    print(resultado3.to_string(index=False))
    
    print("\n\n4. Distribuição por ano e mês:")
    query4 = """
        SELECT 
            ano_bo,
            mes_estatistica,
            COUNT(*) as total_ocorrencias
        FROM ocorrencias_criminais
        GROUP BY ano_bo, mes_estatistica
        ORDER BY ano_bo DESC, mes_estatistica DESC
        LIMIT 20
    """
    resultado4 = conn.execute(query4).fetchdf()
    print(resultado4.to_string(index=False))
    
    conn.close()
    print("\n" + "="*60)
else:
    print("Banco DuckDB não encontrado")

EXEMPLOS DE CONSULTAS SQL - DuckDB

1. Top 10 municípios:
nome_municipio_circunscricao  total_ocorrencias
                     S.PAULO             355118
                    CAMPINAS              25106
                   GUARULHOS              23685
                     S.ANDRE              17053
         S.BERNARDO DO CAMPO              14585
                      OSASCO              14029
              RIBEIRAO PRETO              12876
                    SOROCABA              12715
         S.JOSE DO RIO PRETO              10267
                PRAIA GRANDE               9357


2. Top 10 tipos de crime:
                                                                   rubrica  total_ocorrencias
                                                          Furto (art. 155)             481753
                                                          Roubo (art. 157)             144917
                                                 Lesão corporal (art. 129)             123679
         L

In [21]:
db_duckdb = DATA_PROCESSED_DIR / 'seguranca_publica_sp.duckdb'

if db_duckdb.exists():
    conn = duckdb.connect(str(db_duckdb))
    
    print("="*60)
    print("ANÁLISES AVANÇADAS - DuckDB")
    print("="*60)
    
    print("\n1. Análise por município (Top 10):")
    query_avancada1 = """
        SELECT 
            nome_municipio_circunscricao,
            COUNT(DISTINCT rubrica) as tipos_crime_distintos,
            COUNT(*) as total_ocorrencias,
            COUNT(DISTINCT categoria_crime) as categorias_distintas
        FROM ocorrencias_criminais
        GROUP BY nome_municipio_circunscricao
        ORDER BY total_ocorrencias DESC
        LIMIT 10
    """
    resultado_av1 = conn.execute(query_avancada1).fetchdf()
    print(resultado_av1.to_string(index=False))
    
    print("\n\n2. Distribuição temporal por categoria:")
    query_avancada2 = """
        SELECT 
            ano_bo,
            categoria_crime,
            COUNT(*) as total
        FROM ocorrencias_criminais
        WHERE ano_bo >= 2023
        GROUP BY ano_bo, categoria_crime
        ORDER BY ano_bo DESC, total DESC
    """
    resultado_av2 = conn.execute(query_avancada2).fetchdf()
    print(resultado_av2.to_string(index=False))
    
    print("\n\n3. Crimes com coordenadas geográficas:")
    query_avancada3 = """
        SELECT 
            COUNT(*) as total_registros,
            COUNT(CASE WHEN latitude IS NOT NULL AND longitude IS NOT NULL THEN 1 END) as com_coordenadas,
            ROUND(COUNT(CASE WHEN latitude IS NOT NULL AND longitude IS NOT NULL THEN 1 END) * 100.0 / COUNT(*), 2) as percentual_geolocalizados
        FROM ocorrencias_criminais
    """
    resultado_av3 = conn.execute(query_avancada3).fetchdf()
    print(resultado_av3.to_string(index=False))
    
    conn.close()
    print("\n" + "="*60)
else:
    print("Banco DuckDB não encontrado")

ANÁLISES AVANÇADAS - DuckDB

1. Análise por município (Top 10):
nome_municipio_circunscricao  tipos_crime_distintos  total_ocorrencias  categorias_distintas
                     S.PAULO                     39             355118                     4
                    CAMPINAS                     25              25106                     4
                   GUARULHOS                     23              23685                     4
                     S.ANDRE                     21              17053                     4
         S.BERNARDO DO CAMPO                     23              14585                     4
                      OSASCO                     22              14029                     4
              RIBEIRAO PRETO                     23              12876                     4
                    SOROCABA                     21              12715                     4
         S.JOSE DO RIO PRETO                     23              10267                     4
      

## 11. Resumo Final da Carga

Célula de summary executivo que consolida todas as informações do processo de carga ETL. Apresenta:

- Arquivos gerados e seus tamanhos
- Formatos disponíveis para consumo
- Localização dos dados processados
- Próximos passos sugeridos (análise exploratória, visualização web)
- Instruções para uso dos dados carregados

Este resumo serve como ponto de conclusão do pipeline ETL e orientação para as próximas etapas do projeto de segurança pública.

In [22]:
print("="*70)
print("RESUMO DO PROCESSO DE CARGA")
print("="*70)

print("\nEtapas concluídas:")
print("  1. Carregamento de dados processados")
print("  2. Consolidação de múltiplos períodos")
print("  3. Salvamento em Parquet otimizado")
print("  4. Carga em banco de dados DuckDB")
print("  5. Criação de tabelas agregadas")
print("  6. Validação de integridade")
print("  7. Criação de relatórios e metadados")
print("  8. Exemplos de consultas SQL")

print("\nArquivos e bancos criados:")

print("\n  data/processed/:")
if DATA_PROCESSED_DIR.exists():
    for file in sorted(DATA_PROCESSED_DIR.glob('*')):
        size = file.stat().st_size
        if size > 1024*1024:
            size_str = f"{size/(1024*1024):.2f} MB"
        else:
            size_str = f"{size/1024:.2f} KB"
        print(f"    {file.name:<45} ({size_str})")

print("\n" + "="*70)
print("PROCESSO DE CARGA CONCLUÍDO")
print("="*70)

print("\nDados prontos para:")
print("  - Análises exploratórias avançadas")
print("  - Dashboards e visualizações")
print("  - Machine Learning")
print("  - Integração com ferramentas de BI")
print("  - APIs e aplicações web")

print("\nComo acessar os dados:")
print("  Parquet: pd.read_parquet('data/processed/seguranca_publica_sp_consolidado.parquet')")
print("  DuckDB:  conn = duckdb.connect('data/processed/seguranca_publica_sp.duckdb')")
print("           conn.execute('SELECT * FROM ocorrencias_criminais').fetchdf()")

print("\nPróximo passo:")
print("  Execute o notebook 4_analise_dados.ipynb para análises exploratórias")

print("\n" + "="*70)

RESUMO DO PROCESSO DE CARGA

Etapas concluídas:
  1. Carregamento de dados processados
  2. Consolidação de múltiplos períodos
  3. Salvamento em Parquet otimizado
  4. Carga em banco de dados DuckDB
  5. Criação de tabelas agregadas
  6. Validação de integridade
  7. Criação de relatórios e metadados
  8. Exemplos de consultas SQL

Arquivos e bancos criados:

  data/processed/:
    metadata.json                                 (1.55 KB)
    metadata_carga.json                           (1.77 KB)
    metadata_transformacao.json                   (2.16 KB)
    ocorrencias_agregadas_municipio_crime.parquet (29.67 KB)
    ocorrencias_com_coordenadas.parquet           (11.67 MB)
    ocorrencias_criminais_2025_completo.parquet   (31.93 MB)
    ocorrencias_criminais_2025_transformado.csv   (358.37 MB)
    ocorrencias_criminais_2025_transformado.parquet (32.26 MB)
    seguranca_publica_sp.duckdb                   (100.51 MB)
    seguranca_publica_sp_consolidado.parquet      (32.26 MB)
    sum